In [1]:
import tensorflow as tf
import tensorflow.compat.v1.keras.backend as K
import numpy as np
import pandas as pd
from matplotlib.pyplot import imshow, legend, show, figure
from keras_lr_finder import LRFinder  

print(tf.version.VERSION)
# tf.compat.v1.disable_v2_behavior()
# tf.compat.v1.disable_eager_execution()
tf.compat.v1.disable_v2_behavior()

2.4.1
Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
lesions = {0: "Basal Cell Carcinoma", 
    1: "Lentigo",
    2: "Malignant Melanoma", 
    3: "Melanocytic naevus",
    4: "seborrhoeic keratosis",
    5: "Wart", 
    6: "Actinic Keratosis",
    7: "Squamous Cell Carcinoma",
    8: "Intraepithelial Carcinoma", 
    9: "Pyogenic Granuloma",
    10: "Haemangioma",
    11: "Dermatofibroma"
}

In [3]:
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [4]:
BATCH_SIZE = 22

src_path_train = "dataset-split/train/"
def my_preprocessing_func(img):
    image = np.array(img)
#     image = image / .8
    image = tf.keras.applications.resnet_v2.preprocess_input(image)
    return image

train_generator = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=my_preprocessing_func).flow_from_directory(
    directory=src_path_train,
    subset='training',
    class_mode="sparse",
    batch_size=BATCH_SIZE,
    target_size=(224, 224),
    shuffle=True,
    seed=42
)

Found 53820 images belonging to 12 classes.


In [5]:
src_path_val = "dataset-split/val/"
val_generator = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=my_preprocessing_func).flow_from_directory(
    directory=src_path_val,
    target_size=(224, 224),
    batch_size=BATCH_SIZE,
    class_mode="sparse",
    shuffle=True,
    seed=42
)

Found 6570 images belonging to 12 classes.


In [6]:
# def plots(ims, figsize=(14,6), rows=3, interp=False, titles='lesions'):
#     if type(ims[0]) is np.ndarray:
#         ims = np.array(ims)
#         if (ims.shape[-1] != 3):
#             ims = ims.transpose((0,2,3,1))
#     f = figure(figsize=figsize)
#     cols = len(ims)//rows if len(ims) % 2 == 0 else len(ims)//rows + 1
#     for i in range(len(ims)):
#         sp = f.add_subplot(rows, cols, i+1)
#         sp.axis('Off')
#         if titles is not None:
#             sp.set_title(titles[i], fontsize=16)
#         imshow(ims[i], interpolation=None if interp else 'none')
        
# imgs, labels = next(train_generator)

# plots(imgs, titles=labels)

In [7]:
# src_path_test = "dataset-split/test/"
# test_generator = tf.keras.preprocessing.image.ImageDataGenerator().flow_from_directory(
#     directory=src_path_test,
#     target_size=(224, 224),
#     color_mode="rgb",
#     batch_size=1,
#     class_mode=None,
#     shuffle=False,
#     seed=42
# )

In [8]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)

In [9]:
# imshow(tf.cast(images_val[5], tf.uint8))

In [10]:
import tensorflow_addons as tfa

lr = 0.0003
eps = 10e-3
epochs = 10
momentum = 0.9
wd = 1e-05
gamma = 0.1
ds = train_generator.samples / BATCH_SIZE


model = tf.keras.models.Sequential()
model.add(tf.keras.applications.ResNet152V2(input_shape=(224,224,3), weights='imagenet', include_top=False))
model.add(tf.keras.layers.GlobalAveragePooling2D(name='avg_pool'))
model.add(tf.keras.layers.Dense(12, activation=tfa.activations.sparsemax, name='predictions'))

# model.add(tf.keras.layers.AveragePooling2D((7, 7), name='avg_pool'))
# model.add(tf.keras.layers.Flatten())
# model.add(tf.keras.layers.Dense(12, activation='softmax', name='predictions'))

# lr_scheduling = tf.keras.optimizers.schedules.ExponentialDecay(
#     initial_learning_rate=lr, decay_steps=ds, decay_rate=dr
# )

lr_scheduling = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=lr, decay_steps=ds, decay_rate=gamma, staircase=True
)


# opt = tf.keras.optimizers.Adam(learning_rate=lr, epsilon=eps)
opt = tfa.optimizers.SGDW(learning_rate=lr_scheduling, momentum=momentum, weight_decay=wd)

# loss = tf.keras.losses.sparse_categorical_crossentropy(
#     y_true= , y_pred, from_logits=False, axis=-1
# )

model.compile(optimizer=opt,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy', 'sparse_top_k_categorical_accuracy', tfa.metrics.MultiLabelConfusionMatrix(num_classes=12)])

history = model.fit(train_generator,
                    validation_data = val_generator,
                    epochs=epochs)

Epoch 1/10
2447/2447 [==============================] - ETA: 0s - batch: 1223.0000 - size: 21.9943 - loss: 4.9659 - acc: 0.3103 - sparse_top_k_categorical_accuracy: 0.7378 - Multilabel_confusion_matrix: 22449.9375

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


2447/2447 [==============================] - 1371s 554ms/step - batch: 1223.0000 - size: 21.9943 - loss: 4.9659 - acc: 0.3103 - sparse_top_k_categorical_accuracy: 0.7378 - Multilabel_confusion_matrix: 22449.9375 - val_loss: 4.3522 - val_acc: 0.3151 - val_sparse_top_k_categorical_accuracy: 0.7416 - val_Multilabel_confusion_matrix: 2739.3958
Epoch 2/10
2447/2447 [==============================] - 1316s 538ms/step - batch: 1223.0000 - size: 21.9943 - loss: 4.4879 - acc: 0.3099 - sparse_top_k_categorical_accuracy: 0.7162 - Multilabel_confusion_matrix: 22496.0000 - val_loss: 4.2937 - val_acc: 0.3151 - val_sparse_top_k_categorical_accuracy: 0.7215 - val_Multilabel_confusion_matrix: 2740.5833
Epoch 3/10
2447/2447 [==============================] - 1304s 533ms/step - batch: 1223.0000 - size: 21.9943 - loss: 4.4831 - acc: 0.3099 - sparse_top_k_categorical_accuracy: 0.7115 - Multilabel_confusion_matrix: 22496.8535 - val_loss: 4.2968 - val_acc: 0.3151 - val_sparse_top_k_categorical_accuracy: 0.72

KeyboardInterrupt: 

In [ ]:
# model.save('model/tensorflow-model')

In [ ]:
history.history

In [ ]:
model.metrics_names

In [ ]:
# model.save_weights('model/tensorflow-model-weights')

In [ ]:
try:
    import shap
except Exception as error:
    !pip install {str(error)[17:-1]}
    import shap
except:
    print("Pacote não encontrado")

In [ ]:
print(x.shape[0])

background = x[np.random.choice(x.shape[0], 2)]

# explain predictions of the model on four images
e = shap.DeepExplainer(model, background)
# ...or pass tensors directly
# e = shap.DeepExplainer((model.layers[0].input, model.layers[-1].output), background)


In [ ]:
shap_values = e.shap_values(x)

In [ ]:
# plot the feature attributions
shap.image_plot(shap_values, -x)

In [ ]:
model.save('model/tensorflow-model-low-test')

In [ ]:
new_model = tf.keras.models.load_model('model/tensorflow-model-low-test')

In [ ]:
from PIL import Image
from numpy import asarray
import pandas as pd
import random
test = pd.read_csv('test.csv',index_col=0)
train = pd.read_csv('train.csv',index_col=0)
df_test = pd.DataFrame(test)
test_dataset_filenames = df_test['id'].values.tolist()
test_dataset_filenames_background = np.random.choice(test_dataset_filenames, 10, replace=False)
test_dataset_filenames = np.random.choice(test_dataset_filenames, 5, replace=False)

numpy_images = []
for image in test_dataset_filenames:
    image = Image.open("../input/datasetsplit/" + image)
    image = asarray(image)
    image = tf.keras.applications.resnet_v2.preprocess_input(image)
    numpy_images.append(image)
    
background = []
for image in test_dataset_filenames_background:
    image = Image.open("../input/datasetsplit/" + image)
    image = asarray(image)
    image = image.astype(np.float32)
    image /= 255
    background.append(image)
    
x_pixel_values = []
for image in test_dataset_filenames:
    image = Image.open("../input/datasetsplit/" + image)
    image = asarray(image)
    image = image.astype(np.float32)
    image /= 255
    x_pixel_values.append(image)

background = asarray(background)
x = asarray(numpy_images)
x_pixel_values = asarray(x_pixel_values)
imshow(x_pixel_values[0])

In [ ]:
print(x.shape[0])

# explain predictions of the model on four images
e = shap.DeepExplainer(new_model, background)
# ...or pass tensors directly
# e = shap.DeepExplainer((model.layers[0].input, model.layers[-1].output), background)


In [ ]:
shap_values = e.shap_values(x)

In [ ]:
shap.image_plot(shap_values, x)